<a href="https://colab.research.google.com/github/heinohen/tko_7095_i2hlt/blob/main/week4_exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [4]:
import torch
import transformers
import datasets

from pprint import pprint # Pretty

dataset = datasets.load_dataset('conll2003')

print(dataset)

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [11]:
POS_TAG_NAMES = dataset['train'].features['pos_tags'].feature.names
NER_TAG_NAMES = dataset['train'].features['ner_tags'].feature.names
CHUNK_TAG_NAMES = dataset['train'].features['chunk_tags'].feature.names


Create mappings from names to IDs and back as Python dicts

In [22]:


POS2ID = { n: i for i, n in enumerate(POS_TAG_NAMES) }
ID2POS = { i: n for i, n in enumerate(POS_TAG_NAMES) }

NER2ID = { n: i for i, n in enumerate(NER_TAG_NAMES) }
ID2NER = { i: n for i, n in enumerate(NER_TAG_NAMES) }

CHUNK2ID = { n: i for i, n in enumerate(CHUNK_TAG_NAMES) }
ID2CHUNK = { i: n for i, n in enumerate(CHUNK_TAG_NAMES) }



In [17]:
POS2DESCRIPTION = {
    "CC": "Coordinating conjunction",
    "CD": "Cardinal number",
    "DT": "Determiner",
    "EX": "Existential there",
    "FW": "Foreign word",
    "IN": "Preposition or subordinating conjunction",
    "JJ": "Adjective",
    "JJR": "Adjective, comparative",
    "JJS": "Adjective, superlative",
    "LS": "List item marker",
    "MD": "Modal",
    "NN": "Noun, singular or mass",
    "NNS": "Noun, plural",
    "NNP": "Proper noun, singular",
    "NNPS": "Proper noun, plural",
    "PDT": "Predeterminer",
    "POS": "Possessive ending",
    "PRP": "Personal pronoun",
    "PRP$": "Possessive pronoun",
    "RB": "Adverb",
    "RBR": "Adverb, comparative",
    "RBS": "Adverb, superlative",
    "RP": "Particle",
    "SYM": "Symbol",
    "TO": "to",
    "UH": "Interjection",
    "VB": "Verb, base form",
    "VBD": "Verb, past tense",
    "VBG": "Verb, gerund or present participle",
    "VBN": "Verb, past participle",
    "VBP": "Verb, non-3rd person singular present",
    "VBZ": "Verb, 3rd person singular present",
    "WDT": "Wh-determiner",
    "WP": "Wh-pronoun",
    "WP$": "Possessive wh-pronoun",
    "WRB": "Wh-adverb"
}

In [23]:
import tabulate

e = dataset["train"][12]    # work on the same example

table = []
for token, pos_id, chunk_id, ner_id in zip(e["tokens"], e["pos_tags"], e["chunk_tags"], e["ner_tags"]):
    ner_tag = ID2NER[ner_id]
    chunk_tag = ID2CHUNK[chunk_id]
    pos_tag = ID2POS[pos_id]
    pos_def = POS2DESCRIPTION.get(pos_tag,pos_tag)
    table.append([token, ner_tag, chunk_tag, pos_tag, pos_def])

print(tabulate.tabulate(table,headers=["Token", "NER", "Chunk", "POS", "POS definition"]))

Token     NER    Chunk    POS    POS definition
--------  -----  -------  -----  ------------------------
Only      O      B-NP     RB     Adverb
France    B-LOC  I-NP     NNP    Proper noun, singular
and       O      I-NP     CC     Coordinating conjunction
Britain   B-LOC  I-NP     NNP    Proper noun, singular
backed    O      B-VP     VBD    Verb, past tense
Fischler  B-PER  B-NP     NNP    Proper noun, singular
's        O      B-NP     POS    Possessive ending
proposal  O      I-NP     NN     Noun, singular or mass
.         O      O        .      .


In [24]:
def token_features(tokens, index, window_size):
  # Generate features for token in position `index`in give list of tokens
  features = []

  # Context window start and end
  window_start = max(0, index - window_size)
  window_end = min(index + window_size + 1, len(tokens)) # note: +1 for range in next step!

  for i in range(window_start, window_end): # <--
    offset = i - index # rel pos
    features.append(f'token[{offset}] = {tokens[i]}')

  # Example custom feature: does focus token start with an upper-case letter ?
  if tokens[index][0].isupper():
    features.append('first-letter-capitalized')

  return features

In [25]:
def add_features_to_sentence(sentence):
  # Collect lists of features for all tokens here
  all_features = []

  tokens = sentence['tokens']
  for i in range(len(tokens)):
    all_features.append(token_features(tokens, i, window_size = 3))

  return { 'features': all_features }

In [27]:
for feats in add_features_to_sentence(dataset['train'][12])['features']:
  print(feats)

['token[0] = Only', 'token[1] = France', 'token[2] = and', 'token[3] = Britain', 'first-letter-capitalized']
['token[-1] = Only', 'token[0] = France', 'token[1] = and', 'token[2] = Britain', 'token[3] = backed', 'first-letter-capitalized']
['token[-2] = Only', 'token[-1] = France', 'token[0] = and', 'token[1] = Britain', 'token[2] = backed', 'token[3] = Fischler']
['token[-3] = Only', 'token[-2] = France', 'token[-1] = and', 'token[0] = Britain', 'token[1] = backed', 'token[2] = Fischler', "token[3] = 's", 'first-letter-capitalized']
['token[-3] = France', 'token[-2] = and', 'token[-1] = Britain', 'token[0] = backed', 'token[1] = Fischler', "token[2] = 's", 'token[3] = proposal']
['token[-3] = and', 'token[-2] = Britain', 'token[-1] = backed', 'token[0] = Fischler', "token[1] = 's", 'token[2] = proposal', 'token[3] = .', 'first-letter-capitalized']
['token[-3] = Britain', 'token[-2] = backed', 'token[-1] = Fischler', "token[0] = 's", 'token[1] = proposal', 'token[2] = .']
['token[-3] =

In [28]:
dataset = dataset.map(add_features_to_sentence)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [29]:
pprint(dataset['train'][12])

{'chunk_tags': [11, 12, 12, 12, 21, 11, 11, 12, 0],
 'features': [['token[0] = Only',
               'token[1] = France',
               'token[2] = and',
               'token[3] = Britain',
               'first-letter-capitalized'],
              ['token[-1] = Only',
               'token[0] = France',
               'token[1] = and',
               'token[2] = Britain',
               'token[3] = backed',
               'first-letter-capitalized'],
              ['token[-2] = Only',
               'token[-1] = France',
               'token[0] = and',
               'token[1] = Britain',
               'token[2] = backed',
               'token[3] = Fischler'],
              ['token[-3] = Only',
               'token[-2] = France',
               'token[-1] = and',
               'token[0] = Britain',
               'token[1] = backed',
               'token[2] = Fischler',
               "token[3] = 's",
               'first-letter-capitalized'],
              ['token[-3] = Franc

In [31]:
def flatten(subset):
  # Keys for values to flatten
  keys = ["tokens", "pos_tags", "chunk_tags", "ner_tags", "features"]

  # Initialize to empty lists of tokens etc
  flattened = {k: [] for k in keys}

  # Concaty per-sentence lists of tokens etc
  for sentence in subset:
    for key in keys:
      flattened[key].extend(sentence[key])

  # Rerturn as Dataset object
  return datasets.Dataset.from_dict(flattened)


In [34]:
flattened_dict = {
    'train': flatten(dataset['train']),
    'validation': flatten(dataset['validation']),
    'test': flatten(dataset['test']),
}

flat_dataset = datasets.DatasetDict(flattened_dict)

flat_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'features'],
        num_rows: 203621
    })
    validation: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'features'],
        num_rows: 51362
    })
    test: Dataset({
        features: ['tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'features'],
        num_rows: 46435
    })
})